# LLM을 활용한 데이터셋 구축

최근 대규모 언어 모델(LLM)의 발전으로 고품질의 텍스트 데이터를 프로그래밍 방식으로 생성하는 것이 가능해졌습니다.  
이러한 합성 데이터셋(Synthetic Dataset)은 특정 태스크에 맞는 학습 데이터를 구하기 어려운 경우 매우 효과적인 대안이 됩니다.

LLM을 활용하여 의도 분류(Intent Classification)를 위한 데이터셋을 자동으로 생성하는 것이 목표입니다.

In [ ]:
import os

from dotenv import load_dotenv
from openrouter_llm import create_openrouter_llm

load_dotenv()

llm = model = create_openrouter_llm("openai/gpt-4.1", temperature=0)

### 데이터 생성 파라미터 정의
데이터셋 생성에 필요한 핵심 파라미터를 정의합니다.   
MACRO_INTENTS는 생성할 데이터의 최상위 의도(Intent) 카테고리를 목록화한 것입니다.  
DEFAULT_ROUTER는 각 의도의 기본 복잡도를 설정하여, 추후 생성될 문장의 suggested_model 레이블을 결정하는 규칙으로 사용됩니다.

In [3]:
# --- 사용자 정의: Macro-Intent 목록 ---
MACRO_INTENTS = [
    "계정_관리",  # 계정 생성/로그인/비밀번호 등
    "주문_결제",  # 주문, 결제 수단, 결제 실패
    "배송_문의",  # 배송상태, 배송기간, 추적
    "상품_정보",  # 상품 상세, 재고, 옵션
    "기술_지원",  # 오류, 기능문의, 사용법
    "반품_환불",  # 반품절차, 환불요청
]

# 라우팅 기준: simple -> small model, complex -> large model
# 여기서는 각 Intent에 대해 기본 권장 모델을 지정할 수 있습니다.
DEFAULT_ROUTER = {
    # Intent: "simple_by_default" (True면 일반적으로 아주 작은 소형모델(2B 이하)로 처리 가능)
    "계정_관리": True,
    "주문_결제": False,
    "배송_문의": True,
    "상품_정보": True,
    "기술_지원": False,
    "반품_환불": False,
}

### LLM을 이용한 발화(Utterance) 데이터 생성 함수
LLM을 호출하여 특정 의도(Intent)에 해당하는 예시 문장들을 생성하는 함수 llm_generate_utterances를 정의합니다.  
이 함수는 LLM에게 JSON 배열 형식으로 응답을 요청하는 프롬프트를 구성하고, 반환된 텍스트를 파싱하여 {'text': ..., 'complexity': ...} 형태의 딕셔너리 리스트로 반환합니다.  

In [4]:
async def llm_generate_utterances(intent: str, n: int = 100) -> list[dict]:
    """
    LLM을 사용해 intent별로 예문을 생성합니다.
    반환 형식: List of dicts: {"text":..., "complexity": "simple"/"complex"}
    프롬프트는 LLM에게 JSONL 형식으로 결과를 달라고 요청합니다.
    """

    # "아래의 Parsing을 통과할 수 있도록, 주어진 "형식"에 집중하여 프롬프팅 해주세요.
    # 예시는 답이 될 수 있는 하나일 뿐이고, 구동 시 확인하실 수 있다시피 이 답 또한 생성 성공 100%를 보장하지 않습니다.
    # 만족하실만한 결과물이 나온다면 충분합니다.
    prompt = (
        f"""다음은 사용자와의 대화 중 대주제 '{intent}'에 해당하는 실제 사용자가 말할 법한 예시 문장 {n} 개를 JSON 배열로 생성해 주세요. 
        각 항목은 'text'와 'complexity' 필드를 가지며 'complexity'는 'simple' 또는 'complex' 중 하나로 표기하세요.
        
        [예시 데이터]
        [{"text": "...", "complexity": "simple"}, ...]'
        
        [주의사항]
        제공된 형식에 어긋나는 답변은 하지 않도록 유의해주세요.
        """
    )
    try:
        result = await llm.ainvoke(prompt)
        return result
    except Exception as e:
        print(f"에러 발생: {e}")

In [ ]:
# TODO: Local 에서 진행합니다.
# Question: LLM에서 출력된 데이터를 어떻게 정리해야할까요? 아래의 코드 구조와 프롬프트를 자세히 살펴보고 고민해서 코드를 작성해주세요.

### 추천 모델 할당 로직 정의
생성된 문장의 의도와 복잡도를 기반으로 small 또는 large 모델을 추천하는 로직을 함수로 정의합니다. 이 함수는 DEFAULT_ROUTER 규칙을 참조하여, 복잡도가 complex인 경우는 항상 large 모델을, 그렇지 않은 경우는 기본 설정에 따라 모델을 할당합니다.

In [5]:
# 문제 2: 문장의 의도(intent)와 복잡도(complexity)에 따라 추천 모델('small' 또는 'large')을 할당하는 로직을 구현합니다.
# 'complexity'가 'complex'이면 항상 'large'를 반환해야 합니다.
# 그렇지 않은 경우, DEFAULT_ROUTER 딕셔너리를 참조하여 모델을 결정합니다.

def assign_suggested_model(intent: str, complexity: str) -> str:
    """기본 라우팅 규칙에 따라 suggested_model을 결정합니다."""
    base_small = DEFAULT_ROUTER.get(intent, True)
    # 복잡한 문장은 대형 모델 권장
    if complexity == "complex":
        return "large"
    return "small" if base_small else "large"

---

### 전체 데이터셋 구축 파이프라인
앞서 정의한 함수들을 조합하여 전체 합성 데이터셋을 구축하는 build_dataset 함수를 정의합니다.  
이 함수는 정의된 모든 MACRO_INTENTS에 대해 반복적으로 llm_generate_utterances를 호출하고, 각 결과에 assign_suggested_model을 적용하여 최종 레코드를 생성합니다.  
생성된 데이터는 중복 제거 후 CSV 및 JSONL 파일 형식으로 저장됩니다.  

In [6]:
from pathlib import Path
import csv, json


def build_dataset(
    intents: list[str], per_intent: int = 100, batch_size: int = 10, out_dir: str = "output_dataset"
) -> Path:
    """
    intents: 생성할 intent 목록
    per_intent: intent당 최종 생성 개수
    batch_size: 한 번에 LLM에 요청할 문장 수
    out_dir: 저장 폴더
    """
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)
    records = []

    for intent in intents:
        print(f"Generating for intent: {intent} (target count={per_intent})")
        generated_count = 0
        while generated_count < per_intent:
            current_batch = min(batch_size, per_intent - generated_count)
            examples = llm_generate_utterances(intent, n=current_batch)
            for ex in examples:
                text = ex.get("text")
                complexity = ex.get("complexity", "simple")
                suggested_model = assign_suggested_model(intent, complexity)
                records.append(
                    {
                        "intent": intent,
                        "text": text,
                        "complexity": complexity,
                        "suggested_model": suggested_model,
                    }
                )
            generated_count += current_batch
            print(f"  Generated so far: {generated_count}/{per_intent}")

    # 중복 제거
    unique_texts = set()
    deduped = []
    for r in records:
        key = (r["intent"], r["text"])
        if key not in unique_texts:
            unique_texts.add(key)
            deduped.append(r)

    # 저장: CSV 및 JSONL
    csv_file = out_path / "intent_dataset.csv"
    jsonl_file = out_path / "intent_dataset.jsonl"

    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["intent", "text", "complexity", "suggested_model"])
        writer.writeheader()
        for r in deduped:
            writer.writerow(r)

    with open(jsonl_file, "w", encoding="utf-8") as f:
        for r in deduped:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    print(f"Saved {len(deduped)} examples -> {csv_file}, {jsonl_file}")
    return out_path

### 데이터셋 생성 실행
정의된 build_dataset 함수를 실행하여 실제 데이터셋 생성을 시작합니다. PER_INTENT 변수는 각 의도당 생성할 문장의 수를 지정합니다. 파일이 이미 존재하는 경우, 생성 과정을 건너뛰도록 조건문이 설정되어 있습니다.

In [7]:
INTENTS = MACRO_INTENTS
PER_INTENT = 100
OUT_DIR = "./"
# if certain path exists
if not os.path.exists("./intent_dataset.csv"):
    build_dataset(INTENTS, per_intent=PER_INTENT, out_dir=OUT_DIR)

Generating for intent: 계정_관리 (target count=100)


TypeError: 'coroutine' object is not iterable

### 생성된 데이터 확인
생성된 CSV 파일을 pandas DataFrame으로 로드하여 상위 10개 행을 출력하고, 총 데이터 수를 확인합니다. 이를 통해 데이터가 의도한 형식과 내용으로 정상적으로 생성되었는지 검증합니다. 또한, JSONL 파일의 내용도 일부 출력하여 형식을 확인합니다.

In [8]:
# 셀 5: 생성된 CSV/JSONL 파일을 확인하고 간단히 미리보기
import pandas as pd

csv_path = "intent_dataset.csv"
jsonl_path = "intent_dataset.jsonl"


print("CSV 존재 여부:", bool(pd.io.common.file_exists(csv_path)))
if pd.io.common.file_exists(csv_path):
    df = pd.read_csv(csv_path)
    display(df.head(10))
    print("총 행 수:", len(df))

# JSONL도 확인
if pd.io.common.file_exists(jsonl_path):
    with open(jsonl_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print("JSONL 샘플 (최초 5줄):")
    print("".join(lines[:5]))
    print("총 JSONL 라인 수:", len(lines))

CSV 존재 여부: False


### 데이터셋 로컬 다운로드

생성된 데이터셋 파일(intent_dataset.csv, intent_dataset.jsonl)을 로컬 머신으로 다운로드하는 코드입니다.  

In [9]:
import os
import shutil

# 파일 복사
csv_path = "intent_dataset.csv"
jsonl_path = "intent_dataset.jsonl"

save_dataset_path = "./"

if os.path.exists(csv_path):
    shutil.copy(csv_path, save_dataset_path)
    print(f"Copied {csv_path} to {save_dataset_path}")

if os.path.exists(jsonl_path):
    shutil.copy(jsonl_path, save_dataset_path)
    print(f"Copied {jsonl_path} to {save_dataset_path}")

---

## 파인튜닝을 위한 Huggingface Dataset 활용법 - 데이터 전처리 파이프라인

> Unsloth 환경을 사용하기 때문에, RunPod 에서 진행해야합니다!


1. **원천 로딩 (`01_data_preparation.ipynb`)** – `wicho/kor_3i4k` train/test split을 불러오고 레이블(0-6)을 의도명과 매핑합니다.
2. **Instruction 변환** – `kor3i4k_to_sharegpt` 함수로 "human/assistant" 턴 2개짜리 ShareGPT 포맷을 생성하고, 답변에는 의도 라벨 텍스트만 남깁니다.
3. **Harmony 포맷 표준화** – `standardize_sharegpt` + `tokenizer.apply_chat_template()`로 GPT-OSS Harmony 템플릿을 적용하고 `<|channel|>final` 토큰을 덧붙여 추론 시스템과 호환시킵니다.
4. **데이터 검증** – 템플릿이 올바르게 적용됐는지 샘플 텍스트를 출력하고, 토큰 길이/레이블 누락 여부를 스크립트로 확인합니다.

In [ ]:
from datasets import load_dataset

raw_ds = load_dataset("wicho/kor_3i4k", split="train")

# ClassLabel 이면 names가 있고, 아니면 그냥 None
label_feature = raw_ds.features["label"]
if hasattr(label_feature, "names"):
    id2label = {i: name for i, name in enumerate(label_feature.names)}
else:
    # 혹시 ClassLabel이 아니라면, 그냥 str로 캐스팅
    id2label = None

데이터셋을 GPT-OSS 프롬프트 형식에 맞추기 위해 커스텀 템플릿을 적용합니다.  
GPT-OSS의 독특한 특징은 OpenAI [Harmony](https://github.com/openai/harmony) 포맷을 사용한다는 점입니다.   
Harmony는 대화 구조, 추론 출력(reasoning output), 도구 호출(tool calling)을 지원합니다.

#### Harmony 템플릿 검증
데이터셋에 `<|start|>system`, `<|start|>user`, `<|start|>assistant<|channel|>final<|message|>`, `<|return|>` 토큰이 모두 포함되어 있는지, 의도 라벨이 누락되지 않았는지 간단히 검사합니다.  

In [ ]:
# Harmony 템플릿/레이블 검증
from collections import Counter

REQUIRED_TOKENS = [
    "<|start|>system",
    "<|start|>user<|message|>",
    "<|start|>assistant<|channel|>final<|message|>",
    "<|return|>",
]

issues = []
intent_counter = Counter()

for idx, example in enumerate(dataset):
    text = example["text"]
    for token in REQUIRED_TOKENS:
        if token not in text:
            issues.append((idx, f"missing token: {token}"))
            break
    # 의도 라벨 유무 확인 (assistant 응답 끝에 위치)
    intent_text = text.split("<|start|>assistant<|channel|>final<|message|>")[-1]
    label = intent_text.replace("<|return|>", "").strip()
    if not label:
        issues.append((idx, "missing intent label"))
    else:
        intent_counter[label] += 1

print(f"검증 완료: 총 {len(dataset)}개 샘플")
print(f"의도 분포 상위 5개: {intent_counter.most_common(5)}")
if issues:
    print(f"발견된 이슈 {len(issues)}건 (최초 5개): {issues[:5]}")
else:
    print("모든 샘플이 요구 토큰과 의도 라벨을 포함합니다.")

In [ ]:
def kor3i4k_to_sharegpt(examples):
    convos = []

    for text, label in zip(examples["text"], examples["label"], strict=False):
        if id2label is not None:
            label_str = id2label[int(label)]
        else:
            label_str = str(label)

        # 필요하면 kor_3i4k 전체 label 목록도 instruction에 넣을 수 있음
        # 여기선 간단히 "label_str만 출력하라"고 강하게 제한
        user_msg = (
            "다음 한국어 발화의 화자 의도(label)를 분류해 주세요.\n"
            "정답으로는 해당 label 이름만 한 단어로 출력하세요.\n\n"
            f"발화: {text}"
        )

        assistant_msg = label_str

        convos.append(
            [
                {"from": "human", "value": user_msg},
                {"from": "gpt", "value": assistant_msg},
            ]
        )

    return {"conversations": convos}


sharegpt_ds = raw_ds.map(
    kor3i4k_to_sharegpt,
    batched=True,
    remove_columns=raw_ds.column_names,  # text/label 제거하고 conversations만 남길지 선택
)


In [ ]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(sharegpt_ds)


def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) # 이 부분은 당장 데이터 로드가 어려운데, 어떻게 해결해볼 수 있을까요?
        for convo in convos
    ]
    return {
        "text": texts,
    }


dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)


def add_channel_final(examples):
    fixed_texts = []
    for text in examples["text"]:
        fixed_texts.append(
            text.replace(
                "<|start|>assistant<|message|>",
                "<|start|>assistant<|channel|>final<|message|>",
            )
        )
    return {"text": fixed_texts}


# 이미 채널 태그가 있으면 중복으로 안 바꾸도록 한 번만 체크
if "<|channel|>final" not in dataset[0]["text"]:
    dataset = dataset.map(add_channel_final, batched=True)
dataset


데이터셋을 확인하고 첫 번째 예시가 어떻게 구성되었는지 살펴보겠습니다.

In [ ]:
dataset[0]["text"]

In [ ]:
# TODO: RunPod 환경에서 진행!
# 1) 데이터셋을 로컬에 저장
# 2) datasets 라이브러리를 통해서 로컬에 저장한 데이터를 로드해주세요.